In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import get_mesh_laplacian
# 3 x F (4)
# N x 3
face = torch.tensor([[0, 2, 3, 0], [1, 1, 2, 2], [2, 4, 4, 3]], dtype=torch.long)
pos = torch.tensor([[0, 1, 0], [1, 0, 0], [1, 1, 0], [1, 2, 0], [2, 1, 0]], dtype=torch.float)

data = Data(pos=pos, face=face)
edge_index, edge_weights = get_mesh_laplacian(data.pos, data.face)
row, col = edge_index

row_wise_sum = scatter_add(edge_weights, row, dim=0, dim_size=5)

In [8]:
edge_weights.shape

torch.Size([21])

100%|██████████| 60/60 [00:03<00:00, 19.01it/s]
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
100%|██████████| 60/60 [00:06<00:00,  8.63it/s]


0

tensor(0.1873)

In [1]:
import torch
from pytorch3d.ops import cot_laplacian
from od3d.cv.geometry.objects3d.meshes import Meshes
from od3d.cv.visual.show import show_scene
from pathlib import Path

bunny = Meshes.load_by_name('sphere')
bunny.verts = bunny.verts * 20
# (verts: Tensor, faces: Tensor, eps: float = 1e-12)
# cot_laplacian
l, inv_face_areas = cot_laplacian(verts=bunny.verts.cuda(), faces=bunny.faces.cuda())
l = l / 2.

l_dense = l.to_dense()
row_sum = -l_dense.sum(dim=-1)
l_dense = torch.diagonal_scatter(input=l_dense, src=row_sum)
l = l_dense.to_sparse_coo()

# normalize symmetric
#inv_face_areas = (inv_face_areas ** 0.5)
#l = inv_face_areas * l * inv_face_areas.T 

# normalize row wise
l = inv_face_areas * l

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:

import torch
vals, vects = torch.lobpcg(l.cuda(), k=3, largest=True)
#vects = vects[:, 3:6]
# https://github.com/pyg-team/pytorch_geometric/issues/6037
# https://www.sciencedirect.com/science/article/pii/S0097849309000272
# Discrete Laplace-Beltrami Operators for Shape Analysis and Segmentation 2009 

#l_dense_norm = l.clone().to_dense()
#l_dense_norm = l_dense_norm / l_dense_norm.norm(dim=-1, keepdim=True)

#rgb = torch.mm(l.T, vects).clamp(0, 1)
#rgb = (rgb + 1) / 2.
#rgb = rgb.softmax(dim=-1)
#vects[:, 1:] = 0.
#rgb = (vects * 10).softmax(dim=0)
rgb = (((vects / vects.abs().max(dim=0, keepdim=True)[0]) + 1.) / 2.).clamp(0, 1)
bunny.rgb = torch.nn.Parameter(rgb)
show_scene(meshes=bunny, fpath=Path('bunny.webm'), viewpoints_count=10, fps=1, return_visualization=False)

100%|██████████| 10/10 [00:00<00:00, 27.07it/s]
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
100%|██████████| 10/10 [00:00<00:00, 26.91it/s]


0

In [15]:
import torch
from pytorch3d.ops import cot_laplacian
from od3d.cv.geometry.objects3d.meshes import Meshes
from od3d.cv.visual.show import show_scene
from pathlib import Path

bunny = Meshes.load_by_name('bunny')
bunny.cuda()
bunny.verts = bunny.verts * 20
bunny.verts = bunny.verts - bunny.verts.mean(dim=0, keepdim=True)

In [28]:
from pytorch3d.ops import sample_farthest_points
from pytorch3d.structures import Pointclouds
pts_vals, pts_inds = sample_farthest_points(points=bunny.verts[None,], K=30)
pts_vals = pts_vals[0]
pts_inds = pts_inds[0]

dist_pts = torch.cdist(bunny.verts, pts_vals) # N x K torch.ones(N, N) * torch.inf
# linear
# dist_max = dist_pts.max()
# label_smooth = (1. - (dist_pts / dist_max)) # N x K
# label_smooth = label_smooth / label_smooth.norm(dim=-1, keepdim=True)

# gaussian
dist_nearest_std_factor = 1
dist_nearest_std = (dist_pts**2).min(dim=1, keepdim=True)[0].mean() ** 0.5
label_smooth = (-dist_pts**2 / ((dist_nearest_std_factor * dist_nearest_std)**2)).softmax(dim=-1) # N x K

bunny.rgb = torch.nn.Parameter(label_smooth[:, :3])
show_scene(meshes=bunny, fpath=Path('bunny.webm'), viewpoints_count=30, fps=3, return_visualization=False)

100%|██████████| 30/30 [00:01<00:00, 17.24it/s]
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
100%|██████████| 30/30 [00:01<00:00, 16.55it/s]


0

100%|██████████| 10/10 [00:00<00:00, 17.71it/s]
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
100%|██████████| 10/10 [00:00<00:00, 24.52it/s]


0

In [15]:
pts_vals.shape

torch.Size([3, 3])

In [2]:
N = bunny.verts.shape[0]
dist = torch.ones((N, N)).cuda() * torch.inf
dist_step = torch.cdist(bunny.verts, bunny.verts) #  torch.ones(N, N) * torch.inf
#for i in range(N):
dist = bunny.get_geodesic_dist()

bunny.faces.shape

torch.Size([69451, 3])

In [3]:
e01 = torch.stack([bunny.faces[:, 0], bunny.faces[:, 1]], dim=1)  # (sum(F_n), 2)
e12 = torch.stack([bunny.faces[:, 1], bunny.faces[:, 2]], dim=1)  # (sum(F_n), 2)
e20 = torch.stack([bunny.faces[:, 2], bunny.faces[:, 0]], dim=1)  # (sum(F_n), 2)

# All edges including duplicates.
edges = torch.cat([e12, e20, e01], dim=0)  # (sum(F_n)*3, 2)


In [7]:
from od3d.cv.select import batched_indexMD_fill
connected = torch.zeros((N, N), dtype=bool).cuda()
connected[edges[:, 0], edges[:, 1]] = True
dist_step[~connected] = torch.inf
#fill_vals = torch.ones((N,)).cuda()
#connected = batched_indexMD_fill(inputMD=connected, indexMD=edges, value=fill_vals)


In [10]:
dist = dist_step.clone()